In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns
import pickle as pkl

In [2]:
def seqDist(s1, s2):
    return sum([1 if b1 != b2 else 0 for b1, b2 in zip(s1, s2)])

In [3]:
def getMinDist(start_seqs, target_seqs):
    res = []
    for target_seq in target_seqs:
        dists = []
        for start_seq in start_seqs:
            dists.append(seqDist(start_seq, target_seq))
        res.append(min(dists))
    return pd.Series(res).value_counts()

In [4]:
df = pd.read_csv('/home/jardic/Documents/projects/jk/datasets/datasets_prepped/strc_km.csv', usecols=['cpm', 'varseq'])

In [5]:
with open('splits.pkl', mode='rb')as sf:
    splits = pkl.load(sf)

df_val = df.loc[splits['val']]
df_tst = df.loc[splits['tst']]

df_val = df_val.sort_values('cpm', ascending=False)
df_tst = df_tst.sort_values('cpm', ascending=False)

In [6]:
topN_starting = 10

In [7]:
for sampling in range(0, len(splits['trn'])):
    
    df_trn = df.loc[splits['trn'][sampling]]
    df_trn = df_trn.sort_values('cpm', ascending=False)
    vc = getMinDist(df_trn['varseq'].head(topN_starting).tolist(), df_val['varseq'].tolist())
    
    plt.figure(figsize=(3, 2.5))
    plt.bar(vc.index, vc.values)
    plt.xlabel('Shortest distance from\nstart set')
    sns.despine()
    plt.tight_layout()
    plt.savefig('dists_to_val_trn' + str(sampling) + '.svg')
    plt.close()
    
    plt.figure(figsize=(4, 2.5))
    sns.histplot(df_trn['cpm'], binwidth=5, color='tab:blue', edgecolor='tab:blue', element="step")
    
    plt.yscale('log')
    #plt.xlim(-20, 1000)
    #plt.xticks([0, 250, 500, 750, 1000])
    #plt.ylim(None, 10000000)
    
    # Adjust figure params
    sns.despine()
    plt.xlabel('CPM')
    plt.ylabel('Number of sequences')
    plt.tight_layout()
    plt.savefig('fitness_distribution_trn' + str(sampling) + '.svg')
    plt.close()